## Import stuff

In [75]:
import tensorflow as tf
from keras.losses import categorical_crossentropy, mean_squared_error
from keras import layers

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import logging
import time

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.mixture import BayesianGaussianMixture, GaussianMixture
from sklearn.kernel_approximation import Nystroem
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRFClassifier, XGBRegressor, XGBRFRegressor

logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print(tf.__version__)


def get_type(model):
    return str(type(model)).split(".")[-1].split("'")[0]




2.8.0


## Load training data from csv


In [76]:
train_data = pd.read_csv("train_distances.csv",index_col=0)
train_labels = pd.read_csv("train_labels.txt", header=None)
val_data = pd.read_csv("val_distances.csv",index_col=0)
val_labels = pd.read_csv("val_labels.txt", header=None)

for data in [train_data, val_data]:
    data.drop("anchor_left_distance_mn", axis=1, inplace=True)
    data.drop("anchor_right_distance_mn", axis=1, inplace=True)
    for extractor in ["xl","resnet"]:
        data[extractor] = data["anchor_left_distance_"+extractor]-data["anchor_right_distance_"+extractor]
        data.drop("anchor_left_distance_"+extractor, axis=1, inplace=True)
        data.drop("anchor_right_distance_"+extractor, axis=1, inplace=True)


X_train = np.array(train_data.iloc[:, :].values,  dtype=float)
Y_train = np.array(train_labels.iloc[:, :].values,  dtype=float).reshape(-1)
X_val = np.array(val_data.iloc[:, :].values,  dtype=float)
Y_val = np.array(val_labels.iloc[:, :].values,  dtype=float).reshape(-1)

print("X shape: ", X_train.shape)
print("Y shape: ", Y_train.shape)
print("X shape: ", X_val.shape)
print("Y shape: ", Y_val.shape)



X shape:  (7315, 2)
Y shape:  (7315,)
X shape:  (7476, 2)
Y shape:  (7476,)


In [77]:
train_data

,xl,resnet
0,-0.554846,-0.738040
1,-0.495574,-0.504175
2,-0.122495,-0.710170
3,-0.525400,-0.376821
4,0.509301,0.556689
...,...,...
7310,-0.399301,-0.026181
7311,-0.148462,-0.340682
7312,-0.396075,-0.401896
7313,-0.612196,-0.661628


In [78]:
print(X_train)#[:,4:]

[[-0.5548462 -0.73804  ]
 [-0.4955739 -0.5041751]
 [-0.1224949 -0.7101702]
 ...
 [-0.3960754 -0.4018956]
 [-0.6121962 -0.6616282]
 [-0.6227437 -0.5962763]]


## The function that trains and rates models

In [79]:
#baseline model that just checks the distances
class MajorityVote():
    def fit(self,X,Y):
        a=1
        
    def predict_proba(self,X):
        X_ = X#[:,4:]
        pred = []
        for row in X_:
            score_left = 0
            score_right = 0
            score_left+=row[0]
            score_left+=row[1]
                


            pred.append([0 if score_left>score_right else 1,1 if score_left>score_right else 0])

        pred = np.array(pred)
        print(pred.shape)
        return pred


In [80]:
classifiers = [
    MajorityVote(),
    HistGradientBoostingClassifier(),
    ExtraTreesClassifier(),
    RandomForestClassifier(),
    XGBRFClassifier(),
    XGBClassifier(),
    LGBMClassifier(),
    CatBoostClassifier(logging_level='Silent'),
    # AutoML(task="classification",time_budget=240,ensemble=False,auto_augment=False,verbose=False,estimator_list=['catboost']),
]


best_model = None
best_accuracy = 0

for model in classifiers:
    start_time = time.time()
    model_type = get_type(model)
    print("training model ", model_type)

    model.fit(X_train, Y_train)

    pred = model.predict_proba(X_val)
    # print(pred)
    pred = pred[:,0]

    accuracy = metrics.accuracy_score(Y_val, pred<0.5)
        
    print("accuracy for",model_type,": " , accuracy,", time taken: ",int(time.time() - start_time),"s")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model

print("best model: ", get_type(best_model),"score: ",best_accuracy,"\n")




training model  MajorityVote
(7476, 2)
accuracy for MajorityVote :  0.7683253076511504 , time taken:  0 s
training model  HistGradientBoostingClassifier
accuracy for HistGradientBoostingClassifier :  0.7225789192081327 , time taken:  0 s
training model  ExtraTreesClassifier
accuracy for ExtraTreesClassifier :  0.7227126805778491 , time taken:  0 s
training model  RandomForestClassifier
accuracy for RandomForestClassifier :  0.722043873729267 , time taken:  0 s
training model  XGBRFClassifier
accuracy for XGBRFClassifier :  0.7406367041198502 , time taken:  0 s
training model  XGBClassifier
accuracy for XGBClassifier :  0.7253879079721777 , time taken:  0 s
training model  LGBMClassifier
accuracy for LGBMClassifier :  0.7212413055109684 , time taken:  0 s
training model  CatBoostClassifier
accuracy for CatBoostClassifier :  0.7285981808453719 , time taken:  3 s
best model:  MajorityVote score:  0.7683253076511504 

